In [1]:
import torch
print(torch.__version__)


2.6.0+cu124


## Required Libraries

Before running this notebook, make sure you have installed the following libraries:


In [2]:
!pip install pytorch-metric-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [3]:
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse -f https://data.pyg.org/whl/torch-$(python -c 'import torch; print(torch.__version__)').html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 41.3 MB/s eta 0:00:00


In [1]:
import pickle
import torch
import random
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import math
from math import log

file_path = 'Graph.pkl'


eigen_vector='eigenvector.pkl'

outdegree='outdegree.pkl'

node_to_index_mapping='node_to_index_mapping.pkl'

with open(file_path, 'rb') as file:
    # Load the data from the file
    edge_probability = pickle.load(file)

with open(eigen_vector, 'rb') as file:
    # Load the data from the file
    eigenvector = pickle.load(file)


with open(outdegree, 'rb') as file:
    # Load the data from the file
    outdegree = pickle.load(file)


with open(node_to_index_mapping, 'rb') as file:
    # Load the data from the file
    node_to_index_mapping = pickle.load(file)

def outdegree_graph1():
    list_outdegree=[]
    for key in outdegree:
        list_outdegree.append(outdegree[key])
    max_outdegree=max(list_outdegree)
    min_outdegree=min(list_outdegree)

    dict1_out_degree={}
    list_follower=set()
    for key in edge_probability:
        dict1={}
        for follower in edge_probability[key]:
            list1=[]
            for edge in edge_probability[key][follower]:
                source_node_degree=outdegree[key]
                tail_node_degree=outdegree[follower]
                probability=float(source_node_degree+tail_node_degree)/2
                # tag=edge[0]
                list1.append((probability))
            dict1[follower]=list1
        dict1_out_degree[key]=dict1
    return dict1_out_degree


# augmenting with eigenvector

def eigenvector1():
    list_eigenvector=[]
    for key in eigenvector:
        list_eigenvector.append(eigenvector[key])

    #indegree_average=sum(list_indegree)/len(list_indegree)
    max_eigenvector=max(list_eigenvector)
    min_eigenvector=min(list_eigenvector)
    #print("")

    dict1_eigenvector={}  # this file will be like edge_probablity career having probaboility as outdegree


    for key in edge_probability:
        dict1={}
        for follower in edge_probability[key]:
            list1=[]
            for edge in edge_probability[key][follower]:
                source_node_degree=eigenvector[key]
                tail_node_degree=eigenvector[follower]
                probability=float(source_node_degree+tail_node_degree)/2
                # tag=edge[0]
                list1.append((probability))
            dict1[follower]=list1
        dict1_eigenvector[key]=dict1
    return dict1_eigenvector

## Generates augmented graph data


In [3]:
import pickle
import torch
import random
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import math
from math import log
import random

file_path = 'Graph.pkl'


eigen_vector='eigenvector.pkl'

pagerank_file='page_rank.pkl'
with open(pagerank_file, 'rb') as file:
    pagerank = pickle.load(file)

outdegree_file = 'outdegree.pkl'
with open(outdegree_file, 'rb') as file:
    outdegree = pickle.load(file)


indegree_file = 'indegree.pkl'
with open(indegree_file, 'rb') as file:
    indegree = pickle.load(file)

node_to_index_mapping='node_to_index_mapping.pkl'
import numpy as np

node_features="node_features.pkl"

#print(random_array)
with open('Graph.pkl', 'rb') as file:
    # Load the data from the file
    edge_probability = pickle.load(file)


with open('node_features.pkl', 'rb') as file:
    # Load the data from the file
    node_features = pickle.load(file)


set_all_nodes = []

# Add keys first
for key in edge_probability:
    if key not in set_all_nodes:
        set_all_nodes.append(key)

# Then add followers
for key in edge_probability:
    for follower in edge_probability[key]:
        if follower not in set_all_nodes:
            set_all_nodes.append(follower)


dict_mapping={} # key node id value index id
index=0

for node in set_all_nodes:
    dict_mapping[node]=index
    index=index+1

list_of_nodes = list(set_all_nodes)

list_node_features=[] # 0 index node feature will be first

for key in dict_mapping:
    try:
        list_node_features.append(node_features[key])
    except:
        # list_node_features.append(np.random.normal(loc=0, scale=1, size=100).astype(np.float32))
        list_node_features.append(np.full((100,), 0.5, dtype=np.float32))

# Load fairness score dictionary
with open('node_fairness_scores.pkl', 'rb') as f:
    fairness_score_dict = pickle.load(f)

# Build fairness_of_nodes list aligned with dict_mapping order
fairness_of_nodes = []
for node in dict_mapping:
    fairness_val = fairness_score_dict[node]  # fallback to random if missing
    fairness_of_nodes.append(fairness_val)


outdegree_graph=outdegree_graph1()


eigenvector_graph=eigenvector1()




def func_graph_augmentation(random_augentation_number):
#random_augentation_number=2

    if(random_augentation_number==1):
        dict_outdegree_edge_removed={}
        for key in outdegree_graph:
            dict1={}
            for follower in outdegree_graph[key]:
                list1=[]
                for edge in outdegree_graph[key][follower]:
                    if(edge*0.4>random.uniform(0, 1)):  # 0.5
                    # if(edge[1]*0.4>.2):
                        # tag=edge[0]
                        probability=edge
                        list1.append((probability))

                
                if(len(list1)>0):
                    dict1[follower]=list1
            if(follower in dict1):
                dict_outdegree_edge_removed[key]=dict1
        data_graph=graph_return(dict_outdegree_edge_removed)

        return  data_graph


    if(random_augentation_number==2):

        dict_inf_probablity_removed={}
        for key in edge_probability:
            dict1={}
            for follower in edge_probability[key]:
                list1=[]
                for edge in edge_probability[key][follower]:
                    if(edge*0.8>random.uniform(0, 1)):   # 0.4
                    # if(edge[1]*0.8>.2):
                        # tag=edge[0]
                        probability=edge
                        list1.append((probability))
                if(len(list1)>0):
                    dict1[follower]=list1
            if(follower in dict1):
                dict_inf_probablity_removed[key]=dict1
        data_graph=graph_return(dict_inf_probablity_removed)

        return  data_graph
# create the graph data here######################################################################################

def graph_return(graph):
    list_source=[]
    list_target=[]
    for key in graph:
        for follower in graph[key]:
            list_source.append(dict_mapping[key])
            list_target.append(dict_mapping[follower])
    edge_index = torch.tensor([list_source, list_target], dtype=torch.long)
    node_features = torch.tensor(np.array(list_node_features), dtype=torch.float)
    fairness_tensor = torch.tensor(fairness_of_nodes, dtype=torch.float64)  # Shape: [num_nodes]
    data_graph = Data(
        x=node_features,
        edge_index=edge_index,
        fairness=fairness_tensor,
        node_id=torch.tensor(list(dict_mapping.keys()), dtype=torch.long)
    )
    return data_graph



## **PNAConv-based GNN model** for 500 epochs and stores the learned node embeddings in `embeddings_fairness_mul.pkl`.



In [ ]:

import pickle
import torch
import random
from torch_geometric.data import Data
import torch.optim as optim
#from torch_geometric.nn import GCNConv
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import PNAConv
#import input_graph_creation
import torch.nn.functional as F
import numpy as np
#import input_small_graph_creation
from pytorch_metric_learning.losses import NTXentLoss
import torch_geometric
from torch_geometric.loader import DataLoader
import gc
import psutil
from torch_geometric.nn import GCNConv
from torch_geometric.utils import k_hop_subgraph
# Print memory usage
print(psutil.virtual_memory())
print(torch.__version__)
print(torch_geometric.__version__)
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import PNAConv



class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, perceptron_hidden_dim, degree):
        super().__init__()
        self.conv1 = PNAConv(
            in_channels, hidden_channels,
            ["sum","mean","min","max","var","std"],
            ["identity","amplification","attenuation","linear","inverse_linear"],
            degree
        )
        self.conv2 = PNAConv(
            hidden_channels, out_channels,
            ["sum","mean","min","max","var","std"],
            ["identity","amplification","attenuation","linear","inverse_linear"],
            degree
        )
        self.fc1 = torch.nn.Linear(out_channels, perceptron_hidden_dim)
        self.fc2 = torch.nn.Linear(perceptron_hidden_dim, out_channels)

    def forward(self, data1st_graph, data2nd_graph):
        # ---- First graph ----
        x, edge_index = data1st_graph.x, data1st_graph.edge_index
        x = self.conv1(x, edge_index)     # no edge_attr
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(self.fc1(x))
        x_graph1 = self.fc2(x)

        # ---- Second graph ----
        x, edge_index = data2nd_graph.x, data2nd_graph.edge_index
        x = self.conv1(x, edge_index)     # no edge_attr
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(self.fc1(x))
        x_graph2 = self.fc2(x)

        return x_graph1, x_graph2
# loss_func = NTXentLoss(temperature=0.10)

with open("Graph.pkl", "rb") as f:
    edge_probability = pickle.load(f)


def fairness_contrastive_loss(h1, h2, node_ids1, node_ids2,
                              combined_hops, fairness_scores,
                              alpha_max=0.8, eta=1.0,
                              negative_sample_size=256,
                              temperature=0.1):
    """
    Fairness-aware contrastive loss using embeddings from two augmented graphs (h1, h2).
    h1: embeddings from G1
    h2: embeddings from G2
    node_ids1: node ids corresponding to h1
    node_ids2: node ids corresponding to h2
    """
    device = h1.device
    node_id_to_index_1 = {int(nid): idx for idx, nid in enumerate(node_ids1.tolist())}
    node_id_to_index_2 = {int(nid): idx for idx, nid in enumerate(node_ids2.tolist())}

    total_loss = torch.tensor(0.0, device=device)
    influencer_count = 0

    for u, two_hop_followers in combined_hops.items():
        if u not in fairness_scores or u not in node_id_to_index_1:
            continue

        u_index = node_id_to_index_1[u]
        u_embed = h1[u_index]   # h^G1_u
        F_u = torch.tensor(fairness_scores[u], dtype=torch.float, device=device)

        # ---------- Positive term ----------
        # L⁺_u = - (1 / |NG2(u)|) Σ_{v∈NG2(u)} F(u) · w_uv · log σ(sim(h^G1_u, h^G2_v) / τ)
        L_pos = torch.tensor(0.0, device=device)
        pos_count = 0
        S_u = torch.tensor(0.0, device=device)  # structural dominance accumulator

        for v in two_hop_followers:
            if v not in node_id_to_index_2:
                continue

            try:
                w_uv = edge_probability[u][v][0]  # AIS-based edge weight
            except KeyError:
                w_uv = 1.0

            v_index = node_id_to_index_2[v]
            sim_uv = F.cosine_similarity(u_embed.unsqueeze(0), h2[v_index].unsqueeze(0)).squeeze()

            L_pos += - w_uv * F_u * torch.log(torch.sigmoid(sim_uv / temperature) + 1e-8)
            S_u += torch.clamp(sim_uv, min=0.0)  # accumulate structural dominance
            pos_count += 1

        if pos_count > 0:
            L_pos /= pos_count

        # ---------- Negative term ----------
        # L⁻_u = - (1 / Z⁻_u) Σ_{n ∈ V \ Fu} log(1 - σ(sim(h^G1_u, h^G2_n) / τ))
        all_node_ids_2 = set(node_id_to_index_2.keys())
        neg_candidates = list(all_node_ids_2 - set(two_hop_followers) - {u})
        sampled_negatives = random.sample(neg_candidates, min(negative_sample_size, len(neg_candidates)))

        L_neg = torch.tensor(0.0, device=device)
        for n in sampled_negatives:
            n_index = node_id_to_index_2[n]
            sim_un = F.cosine_similarity(u_embed.unsqueeze(0), h2[n_index].unsqueeze(0)).squeeze()
            L_neg += - torch.log(1 - torch.sigmoid(sim_un / temperature) + 1e-8)

        if len(sampled_negatives) > 0:
            L_neg /= len(sampled_negatives)

        # ---------- Structural dominance ----------
        # S(u) = Σ_{v∈N₂-hop(u)} max(0, sim(h_u, h_v))
        # D(u) = min–max normalize(S(u)) ∈ [0,1]   (here: clamped version)
        D_u = torch.clamp(S_u, 0.0, 1.0)

        # ---------- Adaptive multiplier ----------
        # α(u) = clip(η · D(u) · (1 - F(u)), 0, α_max)
        alpha_u = torch.clamp(eta * D_u * (1 - F_u), 0.0, alpha_max)

        # ---------- Adaptive fairness loss ----------
        # L_fair-adapt(u) = L⁺_u + (1 + α(u)) · L⁻_u
        L_u = L_pos + (1 + alpha_u) * L_neg
        total_loss += L_u
        influencer_count += 1

    # ---------- Final objective ----------
    # L_total = (1 / |U|) Σ_{u∈U} L(u)
    if influencer_count > 0:
        return total_loss / influencer_count
    else:
        return torch.tensor(0.0, device=device)



def nt_xent_loss(z1, z2, temperature=0.1):
    # N = batch size (take min of z1 and z2 in case they differ)
    N = min(z1.size(0), z2.size(0))
    z1, z2 = z1[:N], z2[:N]
    # f = fairness_weights[:N]  # (commented out for now)

    # Concatenate embeddings from both views → shape [2N, D]
    z = torch.cat([z1, z2], dim=0)  # z = [z1; z2]

    # Normalize embeddings so cosine similarity = dot product
    z = F.normalize(z, dim=1)

    # Compute similarity matrix: sim_matrix[i, j] = (z[i] · z[j]) / τ
    sim_matrix = torch.mm(z, z.t()) / temperature  # shape [2N, 2N]

    # Mask for removing self-similarity (diagonal entries)
    mask = torch.eye(2 * N, dtype=torch.bool, device=z.device)

    loss = 0.0
    for idx in range(N):
        # Positive pair similarity: s(z1[i], z2[i]) = exp( (z1[i]·z2[i]) / τ )
        pos_sim = torch.exp(F.cosine_similarity(
            z1[idx].unsqueeze(0), 
            z2[idx].unsqueeze(0)
        ) / temperature)

        # Similarity of z1[i] with all others (exp scaled by τ)
        all_sims_1 = torch.exp(sim_matrix[idx, :])

        # Similarity of z2[i] with all others
        all_sims_2 = torch.exp(sim_matrix[idx + N, :])

        # Combine both views’ similarities
        all_sims = all_sims_1 + all_sims_2

        # Remove self-similarity (don’t compare z1[i] with itself, etc.)
        all_sims = all_sims[~mask[idx]]

        # Denominator: sum over all possible pairs (negatives + positive)
        denom = all_sims.sum() + 1e-8

        # NT-Xent loss for this node:
        # L_i = -log( exp(sim_pos/τ) / Σ_j exp(sim(z_i, z_j)/τ) )
        node_loss = -torch.log(pos_sim / denom)

        # Apply weight (if fairness_weights used later, replace 1 with f[i])
        weight = 1
        loss += (weight) * node_loss

    # Final loss = mean over batch
    return loss / N



def train(train_loader, train_loader2, combined_hops, multi_hop, fairness_scores, λ=1.0,
          alpha_max=5, eta=1.0, negative_sample_size=256, temperature=0.1):
    """
    Training loop using NT-Xent loss + fairness-aware contrastive loss.
    """
    model.train()
    total_loss_list = []

    for data1st_graph, data2nd_graph in zip(train_loader, train_loader2):
        optimizer.zero_grad()

        # Compute embeddings from the model
        h_1, h_2 = model(data1st_graph, data2nd_graph)

        # Match sizes if needed
        if h_2.shape[0] > h_1.shape[0]:
            h_2 = h_2[:h_1.shape[0]]
        else:
            h_1 = h_1[:h_2.shape[0]]


        # ---------- Contrastive-Loss ----------
        loss_ntx = nt_xent_loss(h_1, h_2, temperature=temperature)


        # ---------- Fairness-Aware Contrastive Loss ----------

        loss_fair = fairness_contrastive_loss(
                h_1, h_2,
                node_ids1=data1st_graph.node_id[:h_1.size(0)],
                node_ids2=data2nd_graph.node_id[:h_2.size(0)],
                combined_hops=combined_hops,
                fairness_scores=fairness_scores,
                alpha_max=alpha_max,
                eta=eta,
                negative_sample_size=negative_sample_size,
                temperature=temperature
            )


        # ---------- Total Loss ----------
        total_loss = loss_ntx + λ * loss_fair

        # Backpropagation
        total_loss.backward()
        optimizer.step()

        total_loss_list.append(total_loss.item())

    return sum(total_loss_list) / len(total_loss_list)



def save_embeddings(model, loader1, loader2, output_path_g1="embeddings_graph1.pkl", output_path_g2="embeddings_graph2.pkl"):
    model.eval()
    all_embeddings_g1 = []
    all_embeddings_g2 = []
    all_node_ids_g1 = []
    all_node_ids_g2 = []

    with torch.no_grad():
        for data1, data2 in zip(loader1, loader2):
            # Move to device
            data1 = data1.to(next(model.parameters()).device)
            data2 = data2.to(next(model.parameters()).device)

            # Get embeddings separately
            h1, h2 = model(data1, data2)

            # Truncate to align with node_ids
            min_len1 = min(h1.size(0), data1.node_id.size(0))
            min_len2 = min(h2.size(0), data2.node_id.size(0))

            h1 = h1[:min_len1]
            h2 = h2[:min_len2]

            node_ids1 = data1.node_id[:min_len1]
            node_ids2 = data2.node_id[:min_len2]

            all_embeddings_g1.append(h1.cpu())
            all_embeddings_g2.append(h2.cpu())
            all_node_ids_g1.extend(node_ids1.cpu().numpy())
            all_node_ids_g2.extend(node_ids2.cpu().numpy())

    # Convert to numpy
    final_embeddings_g1 = torch.cat(all_embeddings_g1, dim=0).numpy()
    final_embeddings_g2 = torch.cat(all_embeddings_g2, dim=0).numpy()

    # Save Graph1 embeddings
    embeddings_dict_g1 = {
        "node_ids": all_node_ids_g1,
        "embeddings": final_embeddings_g1,
    }
    with open(output_path_g1, "wb") as f:
        pickle.dump(embeddings_dict_g1, f)
    print(f"Saved {len(all_node_ids_g1)} node embeddings for Graph1 to '{output_path_g1}'")

    # Save Graph2 embeddings
    embeddings_dict_g2 = {
        "node_ids": all_node_ids_g2,
        "embeddings": final_embeddings_g2,
    }
    with open(output_path_g2, "wb") as f:
        pickle.dump(embeddings_dict_g2, f)
    print(f"Saved {len(all_node_ids_g2)} node embeddings for Graph2 to '{output_path_g2}'")


def manual_cluster_split(input_graph1, input_graph2, num_parts=10):
    assert input_graph1.num_nodes == input_graph2.num_nodes, "Node count mismatch!"
    num_nodes = input_graph1.num_nodes
    # node_indices = torch.randperm(num_nodes)
    node_indices = torch.arange(num_nodes)
    clusters_1 = []
    clusters_2 = []

    part_size = (num_nodes + num_parts - 1) // num_parts  # ceiling division
    for i in range(0, num_nodes, part_size):
      idx = node_indices[i:i+part_size]

      sub1 = input_graph1.subgraph(idx)
      sub2 = input_graph2.subgraph(idx)

      sub1.node_id = input_graph1.node_id[idx]
      sub2.node_id = input_graph2.node_id[idx]

      sub1.fairness = input_graph1.fairness[idx]
      sub2.fairness = input_graph2.fairness[idx]

      clusters_1.append(sub1)
      clusters_2.append(sub2)

    return clusters_1, clusters_2

def loader_func(input_graph1, input_graph2):
    # Get matching clusters from both graphs
    cluster_subgraphs_1, cluster_subgraphs_2 = manual_cluster_split(input_graph1, input_graph2, num_parts=10)

    # Create data loaders
    train_loader_1 = DataLoader(cluster_subgraphs_1, batch_size=1, shuffle=False)
    train_loader_2 = DataLoader(cluster_subgraphs_2, batch_size=1, shuffle=False)

    return train_loader_1, train_loader_2







in_channels=100
hidden_channels = 60  # Example value, you can change it
#out_channels = 120  # Example value, you can change it
out_channels = 50
#perceptron_hidden_dim = 250
perceptron_hidden_dim = 60
degree=torch.tensor([240, 328,  79,  39,  23,  12,  11,   7,   6,   5,   7,   3,   1,   0,
          2,   0,   0,   0,   1])

model = Net(in_channels, hidden_channels, out_channels,perceptron_hidden_dim,degree)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

with open("multi_hop_followers.pkl", "rb") as f:
    multi_hop_followers = pickle.load(f)

with open("combined_hops.pkl", "rb") as f:
    combined_hops = pickle.load(f)


with open("node_fairness_scores.pkl", "rb") as f:
    fairness_scores = pickle.load(f)

graph_list=[]
times=1
while(times<3):
    # random_choice = times
    random_choice = np.random.choice([1, 2])
    data_graph=func_graph_augmentation(random_choice)
    graph_list.append(data_graph)
    times=times+1




train_loader,train_loader2=loader_func(graph_list[0],graph_list[1])
for epoch in range(1, 501):

    loss = train(train_loader,train_loader2,combined_hops,multi_hop_followers,fairness_scores)
    print(f"epoch {epoch}, loss: {loss}")

save_embeddings(model, train_loader, train_loader2, output_path_g1="embeddings_graph1.pkl", output_path_g2="embeddings_graph2.pkl")


svmem(total=8455864320, available=2739220480, percent=67.6, used=5716643840, free=2739220480)
2.2.2+cu118
2.6.1
epoch 1, loss: 13.069145679473877
epoch 2, loss: 9.985326337814332
epoch 3, loss: 10.554227733612061
epoch 4, loss: 9.407706451416015
epoch 5, loss: 10.067283296585083
epoch 6, loss: 8.841991424560547
epoch 7, loss: 9.919242000579834
epoch 8, loss: 9.011754989624023
epoch 9, loss: 9.162083148956299
epoch 10, loss: 8.731242227554322
epoch 11, loss: 8.536951398849487
epoch 12, loss: 8.862805032730103
epoch 13, loss: 8.208715057373047
epoch 14, loss: 8.389186143875122
epoch 15, loss: 8.702409839630127
epoch 16, loss: 7.996313190460205
epoch 17, loss: 7.788460636138916
epoch 18, loss: 7.940563821792603
epoch 19, loss: 7.508432340621948
epoch 20, loss: 7.920890235900879
epoch 21, loss: 7.897067213058472
epoch 22, loss: 7.178870820999146
epoch 23, loss: 7.193622398376465
epoch 24, loss: 7.015471267700195
epoch 25, loss: 6.655825614929199
epoch 26, loss: 8.203508043289185
epoch 27, 

In [6]:
import pickle

# Load the pickle file
with open("embeddings_graph1.pkl", "rb") as f:
    data = pickle.load(f)

# Print keys
print("Keys in pickle file:", data.keys())
for i in range(2):
    print(f" embeddings: {data['embeddings'][i]}")


Keys in pickle file: dict_keys(['node_ids', 'embeddings'])
 embeddings: [-0.0163359   0.59953594 -0.01570386  0.03813164 -0.07691559 -0.3168839
 -0.28525218 -0.5654796   0.09474057  0.34046745  0.2759269  -0.13153133
  0.03908022 -0.26498938  0.15061544  0.02190842  0.07657653  0.4277137
  0.06772633 -0.08555333  0.034463   -0.16574213 -0.11050139  0.0276746
 -0.21091281 -0.326487    0.08237393 -0.2976696   0.49293166  0.09948264
 -0.14257279 -0.04852556 -0.22392222 -0.41465214  0.40467486  0.26199389
  0.44463763  0.3767833  -0.38445532  0.10642318 -0.04477096 -0.24914044
  0.4613399  -0.2515414  -0.02934211  0.10974513 -0.01337276 -0.06883903
 -0.06811655  0.16130649]
 embeddings: [ 1.1786621  -3.901099    2.6943412   0.84948534  0.92163324 -0.6989604
 -1.6502064   4.254978   -1.098114    5.6994     -0.76367116 -0.5996207
  2.1090372  -3.220058   -2.7920866   3.3065708   2.829766    1.9585996
 -0.46227753 -0.9545436  -3.2467198   4.6326737   1.1105989   1.2910306
  6.099027   -3.1311

In [7]:
import pickle

# Load the pickle file
with open("embeddings_graph2.pkl", "rb") as f:
    data = pickle.load(f)

# Print keys
print("Keys in pickle file:", data.keys())
for i in range(2):
    print(f" embeddings: {data['embeddings'][i]}")


Keys in pickle file: dict_keys(['node_ids', 'embeddings'])
 embeddings: [-0.0163359   0.59953594 -0.01570386  0.03813164 -0.07691559 -0.3168839
 -0.28525218 -0.5654796   0.09474057  0.34046745  0.2759269  -0.13153133
  0.03908022 -0.26498938  0.15061544  0.02190842  0.07657653  0.4277137
  0.06772633 -0.08555333  0.034463   -0.16574213 -0.11050139  0.0276746
 -0.21091281 -0.326487    0.08237393 -0.2976696   0.49293166  0.09948264
 -0.14257279 -0.04852556 -0.22392222 -0.41465214  0.40467486  0.26199389
  0.44463763  0.3767833  -0.38445532  0.10642318 -0.04477096 -0.24914044
  0.4613399  -0.2515414  -0.02934211  0.10974513 -0.01337276 -0.06883903
 -0.06811655  0.16130649]
 embeddings: [ 1.1786621  -3.901099    2.6943412   0.84948534  0.92163324 -0.6989604
 -1.6502064   4.254978   -1.098114    5.6994     -0.76367116 -0.5996207
  2.1090372  -3.220058   -2.7920866   3.3065708   2.829766    1.9585996
 -0.46227753 -0.9545436  -3.2467198   4.6326737   1.1105989   1.2910306
  6.099027   -3.1311